In [1]:
from ScheduleSimulator import SchedulerSimulation
from generate_requests_v2 import RequestGeneratorV2, RequestInjection
# from scheduler_v2 import Scheduler
from scheduler_v3 import SchedulerV3 as Scheduler
from test_template import PerformanceTest1 as pt1
import random
import json

import pulp as pl
from gurobipy import tuplelist

In [2]:
random.seed(1)
rg = RequestGeneratorV2(0, 15000, 5, 300, 1)
rg.generate_input_params()
rg.generate_single_request_injection(0, 500, 60, 3000)
output = json.dumps(rg.output_to_json())
input_data = json.loads(output)

In [3]:
input_data

{'input_parameters': {'start_time': 0,
  'slice_size': 300,
  'horizon': 15000,
  'resources': {'t0': [{'start': 0, 'end': 15000}]},
  'proposals': {'proposal_0': {'tac_priority': 9},
   'proposal_1': {'tac_priority': 21},
   'proposal_2': {'tac_priority': 12},
   'proposal_3': {'tac_priority': 36},
   'proposal_4': {'tac_priority': 33}}},
 'injections': [{'injection_time': 0,
   'injection_type': 'request',
   'injection_data': {'0': {'windows': {'t0': [{'start': 34, 'end': 464},
       {'start': 1537, 'end': 3439},
       {'start': 6219, 'end': 6386},
       {'start': 7090, 'end': 7993},
       {'start': 9952, 'end': 10675},
       {'start': 11400, 'end': 12489},
       {'start': 12569, 'end': 12922},
       {'start': 13693, 'end': 14636}]},
     'duration': 1884,
     'proposal': 'proposal_2',
     'resID': 0},
    '1': {'windows': {'t0': [{'start': 365, 'end': 416},
       {'start': 501, 'end': 1674},
       {'start': 5200, 'end': 9685},
       {'start': 10642, 'end': 14768}]},
   

In [4]:
sim = SchedulerSimulation(data=input_data)

In [5]:
sim.run_simulation()

In [6]:
sim.display_simulation_results()


Simulation Complete.
Scheduled requests for 't0'
64: 13 - 1463 (1450), Priority = 797.309862141279
419: 1500 - 2630 (1130), Priority = 678.8427133906465
495: 2712 - 4768 (2056), Priority = 1235.8766556247704
490: 4800 - 7413 (2613), Priority = 1574.0925465431421
404: 7500 - 9596 (2096), Priority = 1252.4117257204193
348: 9600 - 10471 (871), Priority = 524.5449394131941
87: 10500 - 12800 (2300), Priority = 1375.1141762493792
170: 12900 - 13789 (889), Priority = 534.9329988455636



In [7]:
sim.last_sched.write_model()

In [8]:
sim.last_sched.load_model()

In [9]:
sim.last_sched.solve_model()

In [10]:
sim.display_simulation_results()


Simulation Complete.
Scheduled requests for 't0'
64: 13 - 1463 (1450), Priority = 797.309862141279
419: 1500 - 2630 (1130), Priority = 678.8427133906465
495: 2712 - 4768 (2056), Priority = 1235.8766556247704
490: 4800 - 7413 (2613), Priority = 1574.0925465431421
404: 7500 - 9596 (2096), Priority = 1252.4117257204193
348: 9600 - 10471 (871), Priority = 524.5449394131941
87: 10500 - 12800 (2300), Priority = 1375.1141762493792
170: 12900 - 13789 (889), Priority = 534.9329988455636



In [6]:
i = 0

while i < len(sim.events):
    current_time = sim.events[i].time

    while True:
        print(i)
        sim.process_event(sim.events[i])

        if i+1 >= len(sim.events):
            break

        if sim.events[i+1].time == current_time:
            i+=1
        else:
            break

        print("HERE")
        requests, resources = sim.prepare_scheduler()
        i+=1

    break

0


In [7]:
sched = Scheduler(sim.now, sim.horizon, sim.slice_size, resources, sim.proposals, requests, verbose=0)

NameError: name 'resources' is not defined

In [ ]:
sched.calculate_free_windows()
sched.build_data_structures()

In [79]:
yik = sched.yik
aikt = sched.aikt

In [81]:
m = pl.LpProblem("Pulp Test", pl.LpMaximize)

requestLocations = tuplelist()
scheduled_vars = []

for i in range(len(yik)):
    r = yik[i]
    var = pl.LpVariable(name=str(i), cat="Binary")
    scheduled_vars.append(var)
    requestLocations.append((str(r[0]), r[1], r[2], r[3], var))

for rid in sched.requests:
    match = requestLocations.select(rid, "*", "*", "*", "*", "*")
    nscheduled = pl.LpAffineExpression({isScheduled: 1 for reqid, wnum, priority, resource, isScheduled in match})
    constraint = pl.LpConstraint(nscheduled, -1, f"one_per_reqid_constraint_{rid}", 1)
    # print(constraint)
    m.addConstraint(constraint)

for s in aikt.keys():
    match = tuplelist()
    for timeslice in aikt[s]:
        match.append(requestLocations[timeslice])
    nscheduled = pl.LpAffineExpression({isScheduled: 1 for reqid, winidx, priority, resource, isScheduled in match})
    constraint = pl.LpConstraint(nscheduled, -1, f"one_per_slice_constraint_{s}", 1)
    # print(constraint)
    m.addConstraint(constraint)

obj_affine = pl.LpAffineExpression({isScheduled: (priority + 0.1/(winidx+1.0)) for req, winidx, priority, resource, isScheduled in requestLocations})
m.setObjective(obj_affine)

In [82]:
m.solve()

1

In [83]:
m.objective.value()

7744.278209650856

In [84]:
[(x, x.value()) for x in m.variables()]

[(0, 0.0),
 (1, 0.0),
 (10, 0.0),
 (100, 0.0),
 (1000, 0.0),
 (1001, 0.0),
 (1002, 0.0),
 (1003, 0.0),
 (1004, 0.0),
 (1005, 0.0),
 (1006, 0.0),
 (1007, 0.0),
 (1008, 0.0),
 (1009, 0.0),
 (101, 0.0),
 (1010, 0.0),
 (1011, 0.0),
 (1012, 0.0),
 (1013, 0.0),
 (1014, 0.0),
 (1015, 0.0),
 (1016, 0.0),
 (1017, 0.0),
 (1018, 0.0),
 (1019, 0.0),
 (102, 0.0),
 (1020, 0.0),
 (1021, 0.0),
 (1022, 0.0),
 (1023, 0.0),
 (1024, 0.0),
 (1025, 0.0),
 (1026, 0.0),
 (1027, 0.0),
 (1028, 0.0),
 (1029, 0.0),
 (103, 0.0),
 (1030, 0.0),
 (1031, 0.0),
 (1032, 0.0),
 (1033, 0.0),
 (1034, 0.0),
 (1035, 0.0),
 (1036, 0.0),
 (1037, 0.0),
 (1038, 0.0),
 (1039, 0.0),
 (104, 0.0),
 (1040, 0.0),
 (1041, 0.0),
 (1042, 0.0),
 (1043, 0.0),
 (1044, 0.0),
 (1045, 0.0),
 (1046, 0.0),
 (1047, 0.0),
 (1048, 0.0),
 (1049, 0.0),
 (105, 0.0),
 (1050, 0.0),
 (1051, 0.0),
 (1052, 0.0),
 (1053, 0.0),
 (1054, 0.0),
 (1055, 0.0),
 (1056, 0.0),
 (1057, 0.0),
 (1058, 0.0),
 (1059, 0.0),
 (106, 0.0),
 (1060, 0.0),
 (1061, 0.0),
 (1062,

In [76]:
for x in m.variables():
    if x.value() == 1:
        yik_id = int(str(x).split("_")[0])
        entry = yik[yik_id]
        print(entry)
        print(entry[-1].internal_start)

[0, 0, 269.72580593689935, 't0', 0, <scheduler_utils.PossibleStart object at 0x000002807FBC9990>]
3578
[14, 26, 186.81260692245945, 't0', 0, <scheduler_utils.PossibleStart object at 0x000002807FD75A50>]
9402
[28, 6, 204.5386053894284, 't0', 0, <scheduler_utils.PossibleStart object at 0x000002800801C350>]
2400
[33, 10, 620.4360362234487, 't0', 0, <scheduler_utils.PossibleStart object at 0x0000028008020450>]
9300
[39, 6, 790.1174795702481, 't0', 0, <scheduler_utils.PossibleStart object at 0x0000028008028190>]
8400
[1, 18, 126.23673949729042, 't0', 0, <scheduler_utils.PossibleStart object at 0x000002807FBA3890>]
10800
[40, 3, 700.0601020891924, 't0', 0, <scheduler_utils.PossibleStart object at 0x0000028008029610>]
2400
[53, 5, 7.008199903366896, 't0', 0, <scheduler_utils.PossibleStart object at 0x0000028008039090>]
12000


In [77]:
type(yik)

list